# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [17]:
data.head()
#data.shape
#data[(data['CustomerID']==98200) & (data['ProductName']=="Vol Au Vents")]

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [19]:
totalQuantity_df = data.groupby(["CustomerID","ProductName"]).agg({"Quantity":"sum"})
totalQuantity_df.head()

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [23]:
#pandas.pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All', observed=False)
pivot = pd.pivot_table(totalQuantity_df, index='ProductName', columns='CustomerID', values='Quantity',aggfunc=sum)
pivot = pivot.fillna(0)
#pivot.shape
pivot.head(3)

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [25]:
distances = pd.DataFrame(1/(1 + squareform(pdist(pivot.T, 'euclidean'))), 
                         index=pivot.columns, columns=pivot.columns)
distances.head(3)

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [26]:
top = distances[33].sort_values(ascending=False)[1:6]
top

CustomerID
264     0.087047
3535    0.087047
3317    0.087047
2503    0.085983
3305    0.085638
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [34]:
#totalQuantity_df.head(2)
top_lista=list(top.index)
top_totalQuantity= totalQuantity_df.loc[top_lista]
top_totalQuantity

Quantity
CustomerID ProductName                             
264        Apricots - Halves                      1
           Apricots Fresh                         1
           Bacardi Breezer - Tropical             1
           Bagel - Plain                          1
           Banana - Leaves                        1
...                                             ...
3535       Tray - 16in Rnd Blk                    1
           Wanton Wrap                            1
           Wine - Blue Nun Qualitatswein          1
           Wine - Magnotta, Merlot Sr Vqa         1
           Yoghurt Tubes                          1

[270 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [40]:
top_tQ_byPoductName=top_totalQuantity.groupby(["ProductName"]).agg({"Quantity":"sum"}).sort_values(by="Quantity",ascending=False)
top_tQ_byPoductName.head(3)

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [54]:
no_purchased= pivot[33][(pivot[33]==0)]
#no_purchased
recommend_product=top_tQ_byPoductName.merge(no_purchased, on='ProductName')[:5]
recommend_product

,Quantity,33
ProductName,,
Butter - Unsalted,3,0.0
Wine - Ej Gallo Sierra Valley,3,0.0
Soup - Campbells Bean Medley,3,0.0
Wine - Blue Nun Qualitatswein,3,0.0
Chicken - Soup Base,2,0.0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [72]:
def recomienda(df):
    diccionario = {}
    customers_id_list= list(set(data['CustomerID']))
    for customer in customers_id_list:
        top=distances[customer].sort_values(ascending=False)[1:6]
        top_lista=list(top.index)
        top_totalQuantity= totalQuantity_df.loc[top_lista]
        top_tQ_byPoductName=top_totalQuantity.groupby(["ProductName"]).agg({"Quantity":"sum"}).sort_values(by="Quantity",ascending=False)
        no_purchased= pivot[customer][(pivot[customer]==0)]
        recommend_product=top_tQ_byPoductName.merge(no_purchased, on='ProductName')[:5]
        diccionario[customer]=recommend_product.index.tolist()
    return diccionario

diccionario_total=recomienda(distances)
#diccionario_total




##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [76]:
df = pd.DataFrame(diccionario_total)
df=df.T
df.head(5)

,0,1,2,3,4
83973,Cocoa Butter,Veal - Slab Bacon,Bandage - Fexible 1x3,Ezy Change Mophandle,Chicken - Wieners
59399,"Shrimp - Baby, Warm Water",Knife Plastic - White,Towels - Paper / Kraft,Nantucket - Pomegranate Pear,Snapple - Iced Tea Peach
92168,Cup - Translucent 7 Oz Clear,"Veal - Brisket, Provimi,bnls",Snapple Lemon Tea,Spinach - Baby,Pop Shoppe Cream Soda
49159,"Pasta - Penne, Rigate, Dry",Cookies - Assorted,Puree - Mocha,Cheese - Parmesan Grated,Cheese - Victor Et Berthold
18441,Soupfoamcont12oz 112con,Anchovy Paste - 56 G Tube,Sprouts - Baby Pea Tendrils,"Lemonade - Natural, 591 Ml",Sausage - Breakfast


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [77]:
distance_cosine = distances = pd.DataFrame(1/(1 + squareform(pdist(pivot.T, 'cosine'))), 
                         index=pivot.columns, columns=pivot.columns)
dic_cosine=recomienda(distance_cosine)
df_cosine = pd.DataFrame(dic_cosine)
df_cosine.T.head(5)


,0,1,2,3,4
83973,Pastry - Choclate Baked,Flour - Pastry,Puree - Passion Fruit,Lettuce - Spring Mix,Broom - Corn
59399,Ezy Change Mophandle,Anchovy Paste - 56 G Tube,Garlic - Peeled,Broom - Corn,Water - Aquafina Vitamin
92168,Wiberg Super Cure,Tuna - Salad Premix,Sage - Ground,Apricots Fresh,Beef - Inside Round
49159,"Turnip - White, Organic",Peas - Frozen,Lamb - Ground,Beer - Blue,Beef - Ground Medium
18441,V8 - Berry Blend,Whmis - Spray Bottle Trigger,Mussels - Cultivated,Ecolab - Lime - A - Way 4/4 L,Milk - 2%
